In [1]:
import sys, os
sys.path.append("../../")

from utils.contract_utils import init_contract
from utils.network_utils import configure_network

from brownie.network import connect, accounts, disconnect
from brownie_tokens import MintableForkToken

In [2]:
ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']
configure_network(
    node_provider_https=f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}",
    network_name='mainnet'
)
connect('mainnet-fork')


Launching 'ganache-cli --port 8545 --gasLimit 12000000 --accounts 10 --hardfork istanbul --mnemonic brownie --fork https://eth-mainnet.alchemyapi.io/v2/AxU0de70ONfvbD-3_pQX0wUlaBK6g3G4 --chainId 1'...


In [3]:
pool_contract_addr = "0x9D0464996170c6B9e75eED71c68B99dDEDf279e8"  # cvxcrv pool addr
pool_contract = init_contract(contract_addr=pool_contract_addr)
pool_contract.name()

'Curve.fi Factory Plain Pool: cvxCRV'

In [4]:
pool_coin_addrs = []
for i in range(10):
    try:
        pool_coin_addrs.append(pool_contract.coins(i))
    except:
        break
pool_coin_addrs

['0xD533a949740bb3306d119CC777fa900bA034cd52',
 '0x62B9c7356A2Dc64a1969e19C23e4f579F9810Aa7']

In [5]:
pool_balances = pool_contract.get_balances()
total_coins = (pool_balances[1]+pool_balances[0])
print(f"coin 0 percentage: {pool_balances[0]/total_coins * 100}")
print(f"coin 1 percentage: {pool_balances[1]/total_coins * 100}")

coin 0 percentage: 41.58234538108825
coin 1 percentage: 58.41765461891174


add balance:

In [10]:
amount = int(pool_balances[0] / 15)  # swap size : 1/15th of cvxcrv pool balance
min_amount = int(0.99 * amount)  # 1% slippage
crv = MintableForkToken(pool_coin_addrs[0])  # crv is the first coin of cvxcrv-f pool
cvxcrv = MintableForkToken(pool_coin_addrs[1])

cvxcrv

<cvxCrvToken Contract '0x62B9c7356A2Dc64a1969e19C23e4f579F9810Aa7'>

In [11]:
cvxcrv._mint_for_testing(accounts[0], amount)

Transaction sent: 0x64b65497f64198dc91bad6ad0e3417cdc28793ab4a8abb336a32526455b28c81
  Gas price: 0.0 gwei   Gas limit: 12000000   Nonce: 2
  cvxCrvToken.transfer confirmed   Block: 13262175   Gas used: 36167 (0.30%)



In [12]:
cvxcrv.approve(pool_contract_addr, amount, {"from": accounts[0]})

Transaction sent: 0x3a1d2ed92b683db2b31a062d3ce41c221dc3e2a65ff8435a5b8364360f9511a6
  Gas price: 0.0 gwei   Gas limit: 12000000   Nonce: 4
  cvxCrvToken.approve confirmed   Block: 13262176   Gas used: 29106 (0.24%)



<Transaction '0x3a1d2ed92b683db2b31a062d3ce41c221dc3e2a65ff8435a5b8364360f9511a6'>

In [13]:
pool_contract.exchange(1, 0, amount, min_amount, {"from": accounts[0]})

Transaction sent: 0x3de1dc34192252e77b3fc8c5d1b7f102352dd5189d32ed8359e7405cff90a0ca
  Gas price: 0.0 gwei   Gas limit: 12000000   Nonce: 5
  Vyper_contract.exchange confirmed   Block: 13262177   Gas used: 93998 (0.78%)



<Transaction '0x3de1dc34192252e77b3fc8c5d1b7f102352dd5189d32ed8359e7405cff90a0ca'>

In [14]:
pool_balances = pool_contract.get_balances()
total_coins = (pool_balances[1]+pool_balances[0])
print(f"coin 0 percentage: {pool_balances[0]/total_coins * 100}")
print(f"coin 1 percentage: {pool_balances[1]/total_coins * 100}")

coin 0 percentage: 39.51413481926821
coin 1 percentage: 60.48586518073179


In [15]:
disconnect()

Terminating local RPC client...
